In [56]:
import pandas as pd
from sklearn.metrics import roc_auc_score
import numpy as np

In [103]:
def eval_auc_every_coupon(_pred, _label):
    """
    评价 auc：根据每一组 Coupon_id 分别计算 auc 然后求平均
    :param _pred: 预测的 Dataframe，包含 Coupon_id 和 Probability
    :param _label: 真实的 label 序列
    :return:
    """
    _csv = pd.concat([_pred[['Coupon_id', 'Probability']], pd.Series(_label, name='Truth')], axis=1)
    aucs = []
    for idx, g in _csv.groupby('Coupon_id'):
        if len(np.unique(g['Truth'])) == 1:
            # 如果真实标签全是一种类型(大概率全是0，反例)，那么就没有 auc 了，
            # 即这种优惠券一张也没有被使用，自然就没有查准率，查全率之说了
            aucs.append(0.5)
            continue
        auc = roc_auc_score(g['Truth'], g['Probability'])
        aucs.append(auc)
    return np.mean(aucs)

def eval_auc(_pred, _label):
    """
    评价 auc：在全局上计算 auc
    :param _pred: 预测的 Dataframe，包含 Coupon_id 和 Probability
    :param _label: 真实的 label 序列
    :return:
    """
    _csv = pd.concat([_pred[['Coupon_id', 'Probability']], pd.Series(_label, name='Truth')], axis=1)
    return roc_auc_score(_csv['Truth'], _csv['Probability'])

In [104]:
pred = pd.read_csv('./submit.csv', names=('User_id', 'Coupon_id', 'Date_received', 'Probability'))
label = [0, 1] * (len(pred) // 2)

In [105]:
eval_auc_every_coupon(pred, label), eval_auc(pred, label)

(0.5140186881991474, 0.5026239454740827)